Постройте классификатор спама
Загрузите примеры спама и нормальных сообщений из наборов
данных https://spamassassin.apache.org/old/publiccorpus/
Распакуйте наборы данных и ознакомьтесь с форматом данных.
Разделите наборы данных на обучающий набор и испытательный
набор.
Создайте конвейер подготовки данных для преобразования каждого
почтового сообщения в вектор признаков. Ваш конвейер подготовки
данных должен 

+ трансформировать почтовое сообщение в вектор, который указывает на наличие или отсутствие каждого возможного слова. Например, если все почтовые сообщения содержат только четыре слова, “Hello”, “how”, “are”, “you”, тогда сообщение “Hello you Hello Hello you” было бы преобразовано в вектор [1, 0, 0, 1], или [3, 0, 0, 2], если вы предпочитаете подсчитывать количество вхождений каждого слова.

Можете добавить гиперпараметры к своему конвейеру подготовки
данных, чтобы управлять тем

+ производить ли разбор заголовков почтовых сообщений
+ приводить ли символы каждого сообщения к нижнему регистру
+ удалять ли символы пунктуации 
+ замещать ли все URL строкой “URL”, заменять ли все числа строкой “NUMBER”
+ выполнять ли морфологический поиск (stemming), т.е. исключение окончаний слов (для этого доступно несколько библиотек Python).

Проведите тестирование нескольких классификаторов. Уровень
точности и полноты должен быть не менее 0.94.

In [1]:
import numpy as np
import pandas as pd
import os
import re
import tarfile
import urllib.request
import sklearn.utils

from bs4 import BeautifulSoup
from pathlib import Path
from nltk.corpus import stopwords
from sklearn import tree
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_extraction import text
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
easy_ham_path = "C:/Users/Morzhig/Labs/ML/easy_ham/"
hard_ham_path = "C:/Users/Morzhig/Labs/ML/hard_ham/"

spam_path = "C:/Users/Morzhig/Labs/ML/Spam/"

easy_ham_filenames = [easy_ham_path + f for f in sorted(os.listdir(easy_ham_path)) if len(f) > 20]
hard_ham_filenames = [hard_ham_path + f for f in sorted(os.listdir(hard_ham_path)) if len(f) > 20]

ham_filenames = easy_ham_filenames + hard_ham_filenames

spam_filenames = [spam_path + f for f in sorted(os.listdir(spam_path)) if len(f) > 20]

print("Ham files :", len(ham_filenames))
print("Spam Files :" ,len(spam_filenames))

Ham files : 2750
Spam Files : 500


In [3]:
import email
import email.policy

def load_email(filepath):
    with open(filepath, "rb") as f:
        msg = email.message_from_bytes(f.read(), policy=email.policy.default)
        
    subject = msg.get('Subject')
    
    body = ""
    if msg.is_multipart():
        for part in msg.get_payload():
            if part.get_content_type() == "text/plain":
                body = part.get_payload()
                break
    else:
        body = msg.get_payload()
    
    body = BeautifulSoup(body, 'lxml').get_text()
    
    return subject, body

ham_emails = [load_email(filepath) for filepath in ham_filenames]
spam_emails = [load_email(filepath) for filepath in spam_filenames]

C:\Users\Morzhig\AppData\Local\Temp\ipykernel_6332\3146820487.py:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  body = BeautifulSoup(body, 'lxml').get_text()
C:\Users\Morzhig\AppData\Local\Temp\ipykernel_6332\3146820487.py:19: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  body = BeautifulSoup(body, 'lxml').get_text()


In [4]:
X = ham_emails + spam_emails
y = [0] * len(ham_emails) + [1] * len(spam_emails)

X, y = sklearn.utils.shuffle(X, y, random_state=42)

print(X[1335])

('[����] ���� �ߴ� ���� Best 5 & �ڰ��� ���� ��ǳ', "\n\n\n요즘 뜨는 직종 Best 5 : 금융 / IT / 방송 / 뷰티 / 인테리어\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n 요즘 뜨는 직종 Best\n5 : 금융 / IT / 방송 / 뷰티 / 인테리어 \n\n\n\n20대 젊은이들이 뽑은 '일하고 싶은\n분야'로 금융, IT, 방송, 뷰티/패션, 인테리어/건축분야가 선정되었다.\n이들 중에서도 금융분야는\n급여가 상대적으로 높고 주5일제 근무에 앞서있는 '외국계 은행'이, IT분야는 '게임'분야가, 방송분야는 요즘\n신세대 대학생들이 가장 선호하는 직업인 홈쇼핑의 '쇼핑호스트'와 케이블방송의 'VJ'를, 뷰티/패션분야는 美에\n대한 관심고조로 인해 '피부관리사' 또는 '스타일리스트', 인테리어/건축분야에서는 리모델링 붐에 힘입어\n'인테리어'쪽이 높은 순위로 꼽혔다.\n그럼, 이처럼 높은\n주가를 구가하고 있는 직종에 근무하기 위해서는 어떤 자격요소를 갖추는 것이 좋을까?본격적인 여름이\n시작되는 이때, 뜨는 직종으로의 확실한 준비를 해보자.\n 금융 : 98년 말 IMF가 시작될 무렵만 해도\n은행/보험/증권계의 한파는 그 어느분야보다도 치열했고 무차별적인 구조조정으로 인해 많은 직장인들이 거리로\n내몰리다시피 되었었다. 하지만 2002년 현재, 카드사의 호황을 시작으로 은행권, 증권, 보험분야의 경력직\n수요가 많아지고 있으며, 신입직의 채용 또한 많아지고 있다. 올 하반기까지 인력시장은 맑음\n상태.\n금융분야는 자격증이 있으면\n있을수록 좋다. 금융권에서 유용하게 쓰이는 자격증으로는 투자상담사, 금융자산관리사(FP),\n재무위험관리사(FRM) 등. 특히 증권사에 근무할 이들이라면 투자상담사나 금융자산관리사의 자격증은 기본이 되고\n있으니 반드시 취득해두도록 하자.  시험일정\n보기\n IT : 올해들면서 IT/인터넷 업계의\n수익구조가 좋아지고 있으며, 인터넷 분야는 토종인터넷 

In [5]:
import spacy

nlp = spacy.load("en_core_web_sm")

def transform(remove_header=True, remove_num=True, remove_url=True, 
              remove_punc=True, lower_case=True, remove_paragraphs=True, remove_endings=True):
    global X
    
    if remove_header:
        X = [email[1] for email in X]
        
        if remove_num:
            X = [re.sub(r'\d+', 'num', email) for email in X]

        if remove_url:
            X = [re.sub(r'https?://\S+|www\.\S+', 'URL', email) for email in X]

        if remove_punc:
            X = [re.sub(r"[^\w\s]|_", ' ', email) for email in X]

        if lower_case:
            X = [email.lower() for email in X]

        if remove_paragraphs:
            X = [email.replace('\n', ' ') for email in X]

        if remove_endings:
            X_lemmatized = []

            for email in X:
                doc = nlp(email)

                lemmas = [token.lemma_ for token in doc]
                lemmatized_email = ' '.join(lemmas)

                X_lemmatized.append(lemmatized_email)
                X = X_lemmatized

        X = [re.sub(r'[^\x00-\x7F]+', 'glyph', email) for email in X]
        
transform()

In [6]:
print(X[1335])

    glyph glyph glyph good num    glyph    it    glyph    glyph    glyph                      glyph glyph glyph good num    glyph    it    glyph    glyph    glyph      numglyph glyph glyph   glyph glyph glyph glyph glyph   it   glyph   glyph glyph   glyph glyph glyph   glyph glyph glyph glyph glyph glyph glyphnumglyph glyph glyph   glyph glyph glyph   itglyph   glyph glyph   glyph glyph glyph glyph glyph glyph glyph glyph   glyph glyph glyph   vj glyph   glyph glyph glyph glyph glyph glyph   glyph   glyph   glyph    glyph glyph glyph glyph glyph   glyph glyph glyph glyph glyph   glyph   glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph   glyph glyph glyph glyph glyph    glyph    numglyph glyph imfglyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph glyph   glyph numglyph glyph   glyph glyph glyph glyph   glyph   glyph glyph glyph glyph glyph   glyph glyph glyph glyph glyph   glyph

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

CountVectorizer = CountVectorizer()

X_countvectorized = CountVectorizer.fit_transform(X)

In [8]:
pd.DataFrame(X_countvectorized.toarray(), columns=CountVectorizer.get_feature_names_out())

,aa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplfnumfhxl,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacnummmvznumcjnumzld,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacunumwlcunumwmnumdlo,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabcnumfudnumhgknumxt,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplfnumfhxlnummh,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaafcvwyfk,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplfnumfhxlnummhyv,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacnlzcunnumljnumfetbkvts,...,zzrnumxunumrtijnpzcejnumjarjnumvkymlnumnnumjdxq,zzvrulbbwiprjxnum,zzz,zzzason,zzzickeletto,zzzlist,zzzz,zzzzason,zzzzcc,zzzzteana
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_countvectorized, y,  test_size=0.20, random_state=0)

print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))

(2600, 34909) 2600
(650, 34909) 650


In [10]:
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy for Decision Tree: {accuracy}\nPrecision for Decision Tree: {precision}\nRecall for Decision Tree: {recall}')

Accuracy for Decision Tree: 0.943076923076923
Precision for Decision Tree: 0.8073394495412844
Recall for Decision Tree: 0.8461538461538461


In [11]:
sgd = SGDClassifier(random_state=42, max_iter=100, alpha=0.001)
sgd = sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy for SGD: {accuracy}\nPrecision for SGD: {precision}\nRecall for SGD: {recall}')

Accuracy for SGD: 0.9815384615384616
Precision for SGD: 0.9423076923076923
Recall for SGD: 0.9423076923076923


In [12]:
forest = RandomForestClassifier(n_estimators=200)
forest = forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy Random Forest: {accuracy}\nPrecision for Random Forest: {precision}\nRecall for Random Forest: {recall}')

Accuracy Random Forest: 0.9692307692307692
Precision for Random Forest: 0.9772727272727273
Recall for Random Forest: 0.8269230769230769


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVectorizer = TfidfVectorizer()

X_tfidfvectorized = TfidfVectorizer.fit_transform(X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidfvectorized, y,  test_size=0.20, random_state=0)

print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))

(2600, 34909) 2600
(650, 34909) 650


In [26]:
dtc = tree.DecisionTreeClassifier(max_depth=10)
dtc = dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy for Decision Tree: {accuracy}\nPrecision for Decision Tree: {precision}\nRecall for Decision Tree: {recall}')

Accuracy for Decision Tree: 0.9553846153846154
Precision for Decision Tree: 0.8640776699029126
Recall for Decision Tree: 0.8557692307692307


In [16]:
sgd = SGDClassifier(random_state=42, max_iter=100, alpha = 0.0001)
sgd = sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy for SGD: {accuracy}\nPrecision for SGD: {precision}\nRecall for SGD: {recall}')

Accuracy for SGD: 0.9892307692307692
Precision for SGD: 0.9619047619047619
Recall for SGD: 0.9711538461538461


In [17]:
forest = RandomForestClassifier(n_estimators = 300)
forest = forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy for Decision Tree: {accuracy}\nPrecision for Decision Tree: {precision}\nRecall for Decision Tree: {recall}')

Accuracy for Decision Tree: 0.963076923076923
Precision for Decision Tree: 0.9761904761904762
Recall for Decision Tree: 0.7884615384615384


In [27]:
clf = MLPClassifier(random_state=42, max_iter=100).fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy for Decision Tree: {accuracy}\nPrecision for Decision Tree: {precision}\nRecall for Decision Tree: {recall}')

Accuracy for Decision Tree: 0.9892307692307692
Precision for Decision Tree: 0.9532710280373832
Recall for Decision Tree: 0.9807692307692307
